In [1]:
# import modules
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
# import data
df = pd.read_csv(os.getcwd() + "/data/breast-cancer.csv").drop(["id", "Unnamed: 32"], 1)
df.head(5)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    object 
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

In [4]:
# one-hot encode target variable
df_enc = pd.get_dummies(df, drop_first = True)
df_enc.head(5)

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis_M
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1


In [5]:
# define dependent and independent variables
X = df_enc.drop(["diagnosis_M"], 1)
Y = df_enc["diagnosis_M"]

In [6]:
# standardize training data
X = (X - X.mean()) / np.std(X)

In [7]:
# logistic regression from scratch using numpy
class LogisticRegression:
    def __init__(self, eta = 0.001, n_iters = 1000):
        self.eta = eta
        self.n_iters = n_iters
        self.theta = None
        self.bias = None
        
    def fit(self, X, y):
        # initialize the model parameters
        n_samples, n_features = X.shape
        self.theta = np.zeros(n_features)
        self.bias = 0
        
        # gradient decent
        for i in range(self.n_iters):
            # linear model
            linear_model = np.dot(X, self.theta) + self.bias
            # sigmoid function
            proba = self.sigmoid(linear_model)
            # update parameters using derivative of loss function
            dtheta = (1 / n_samples) * np.dot(X.T, (proba - y))
            dbias = (1 / n_samples) * np.sum(proba - y)
            self.theta -= self.eta * dtheta
            self.bias -= self.eta * dbias
            
    def predict(self, X):
        # linear model
        linear_model = np.dot(X, self.theta) + self.bias
        # sigmoid function
        proba = self.sigmoid(linear_model)
        y_hat = [1 if i > 0.5 else 0 for i in proba]
        return y_hat
    
    def score(self, X, y):
        # linear model
        linear_model = np.dot(X, self.theta) + self.bias
        # sigmoid function
        proba = self.sigmoid(linear_model)
        y_hat = [1 if i > 0.5 else 0 for i in proba]
        # compute accuracy
        accuracy = np.sum(y_hat == y) / len(y)
        return accuracy
        
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

In [8]:
model = LogisticRegression()
model.fit(X, Y)
model.score(X, Y)

0.9560632688927944